In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://fr.trustpilot.com/review/www.cdiscount.com'
html = requests.get(url)
html

<Response [200]>

In [16]:
# transformation de la page en un document exploitable
# Selon le parsers choisi (html.parser, lxml, lxml-xml, xml, html5lib)
soup = BeautifulSoup(html.text, 'html.parser')

# affichage de manière agencé avec la methode pretiffy
soup.prettify()

# Chercher à partir de la valeur d'une balise tous les éléments d'un commentaire
comment = soup.find_all('div', {"class" : "styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ"})
print(comment[2].text)

SGSerge Gardeil1 avisFRll y a 2 joursAchat d'un téléphone redmi note11 en…Achat d'un téléphone redmi note11 en aout Le téléphone est d'origine anglophone et ne prend pas les mises à jour et garde la langue anglaise malgré les réglages géographiques.Nous sommes le 11 novembre 2022 après renvoi courant début octobre de l'appareil et des promesses de remboursement des sommes déjà versées ( 24 messages d’échange et plusieurs appels au service client Cdisount)Le service client est incapable d'effectuer le remboursement sur le compte ayant servi à l'achat.Pourtant déjà réaliser lors d'achats précédents.Je n'ai à ce jour plus de téléphone ni remboursementJe déconseille vivement ce site qui mène en bateau les clients fidèles pourtant depuis 15 ansDate de l'expérience: 11 novembre 2022


In [74]:
# chercher dans la balise précédente les sous balises correspondant au pays, à la note et au commentaire

pays = comment[1].find('div', {"class" : "typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua"}).span.text
print(pays)

note = comment[1].find('div', {"class" : "styles_reviewHeader__iU9Px"})['data-service-review-rating']
print(note)

titre_commentaire = comment[1].find({"h2" : "typography_heading-s__f7029 typography_appearance-default__AAY17"}).text
print(titre_commentaire)

commentaire = comment[1].find('section', {"class" : "styles_reviewContentwrapper__zH_9M"}).text
print(commentaire)

FR
1
SAV Cdiscount catastrophique
ll y a 4 joursSAV Cdiscount catastrophiqueune insatisfaction complète pour le service après vente de Cdiscount qui malgré la loi refuse de faire son travail de changement de pièce sous prétexte que leur réparateur agréé n'a pas trouver la panne. Cette même panne qui à été constaté par l'un des réparateurs de la marque elle-même. Je vous déconseille fortement de passer par eux pour acheter vos produits. Cela va faire une mois maintenant que j'attends et que je bataille pour que le changement soit fait et pour le moment sans résultat positif pour moi. Je me retrouve avec un appareil qui  plante régulièrement.Date de l'expérience: 10 novembre 2022


In [99]:
from time import time
t0 = time()

# recherche sur plusieurs pages
nb_pages = 6206
liste_url = []
page = 1
while page <= nb_pages :
    liste_url.append('https://fr.trustpilot.com/review/www.cdiscount.com?page='+str(page))
    page = page + 1

liste_pays = []
liste_note = []
liste_titre_commentaire = []
liste_commentaire = []

for page in liste_url:
    comment = soup.find_all('div', {"class" : "styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ"})
    soup = BeautifulSoup(html.text, 'html.parser')
    for nb_avis in range(len(comment)):
        liste_pays.append(comment[nb_avis].find('div', {"class" : "typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua"}).span.text)
        liste_note.append(comment[nb_avis].find('div', {"class" : "styles_reviewHeader__iU9Px"})['data-service-review-rating'])
        liste_titre_commentaire.append(comment[nb_avis].find({"h2" : "typography_heading-s__f7029 typography_appearance-default__AAY17"}).text)
        liste_commentaire.append(comment[nb_avis].find('section', {"class" : "styles_reviewContentwrapper__zH_9M"}).text)

dico = {'pays' : liste_pays, 'note' : liste_note, 'titre' : liste_titre_commentaire, 'commentaire' : liste_commentaire}
df = pd.DataFrame(dico)

print('le temps de calcul est de {:.2f} secondes'.format(time()-t0))

le temps de calcul est de 532.31 secondes


In [100]:
df

,pays,note,titre,commentaire
0,FR,1,Je me suis fais voler mon colis par le livreur,ll y a 4 joursJe me suis fais voler mon colis ...
1,FR,1,SAV Cdiscount catastrophique,ll y a 4 joursSAV Cdiscount catastrophiqueune ...
2,FR,1,Achat d'un téléphone redmi note11 en…,ll y a 2 joursAchat d'un téléphone redmi note1...
3,FR,1,SAV horrible,ll y a 5 joursSAV horribleJe vais faire un ret...
4,FR,2,Je suis fortement déçu,ll y a 5 joursSur invitationJe suis fortement ...
...,...,...,...,...
124115,FR,1,Très mauvaise expérience,ll y a 5 joursTrès mauvaise expérienceTrès mau...
124116,FR,1,Qualité en baisse,ll y a 4 joursQualité en baisseCe site de e-co...
124117,FR,5,Commande très satisfaisante,Il y a 4 heuresSur invitationCommande très sat...
124118,FR,1,Dégâts lors de la livraison d'une…,ll y a 3 joursDégâts lors de la livraison d'un...


In [101]:
df.to_csv('datasets/SatisfactionClients/cdiscount.csv')